In [1]:
import torch
import os
import sys
from pathlib import Path
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('../utils/')
from configuration import *
from utils import *
import FigureConfig as FC
import pNN_Power_Aware as pNN

args = parser.parse_args([])

In [3]:
num_seeds = 10
num_dataset = 13

In [4]:
result = torch.load('result.matrix')
result.shape

torch.Size([13, 10, 51, 8])

In [5]:
import numpy as np

target = [[ 200.,  240.,  400.,  500.,  300.,  300.,  200.,  300.,  600.,  280.,  340.,  280.,  240.],
          [ 400.,  480.,  800., 1000.,  600.,  600.,  400.,  600., 1200.,  560.,  680.,  560.,  480.],
          [ 600.,  720., 1200., 1500.,  900.,  900.,  600.,  900., 1800.,  840., 1020.,  840.,  720.],
          [ 800.,  960., 1600., 2000., 1200., 1200.,  800., 1200., 2400., 1120., 1360., 1120.,  960.]]
target = np.array(target)

gammas = np.linspace(0, 1, 51)

In [6]:
def calculate_hessian_eigenvalues(model, dataloader, loss_function):
    eigenvalues = {}

    model.train()
    
    for images, labels in dataloader:
        
        # 前向传播
        outputs = model(images)
        loss = loss_function(model, images, labels)
        
        # 计算梯度
        model.zero_grad()
        loss.backward(create_graph=True)
        
        for n, p in model.named_parameters():
            if p.requires_grad:
                grad = p.grad.view(-1)
                hessian = torch.zeros(grad.size(0), grad.size(0))
                # 计算Hessian的每个元素
                for i in range(len(grad)):
                    grad2 = torch.autograd.grad(grad[i], p, retain_graph=True, allow_unused=True)[0].view(-1)
                    if grad2 is not None:
                        hessian[i, :] = grad2
                
                # 计算Hessian的特征值
                eigvals = torch.linalg.eigvals(hessian)
                eigenvalues[n] = eigvals
    
    return eigenvalues

In [7]:
values = []

for ds in range(num_dataset):
    values_ds = []
    
    args.DATASET = ds
    train_loader, datainfo = GetDataLoader(args, 'train', path='../dataset/')
    power = result[ds,:,:,7]
    acc = result[ds,:,:,6]
    
    print(datainfo)
    
    for i in range(4):
        values_target = []
        
        t = target[i, ds]
        
        valid = power < t
        valid_acc = acc[valid]
        best_acc = valid_acc.max()
        seed = torch.where(acc == best_acc)[0][0].item()
        gamma_idx = torch.where(acc == best_acc)[1][0].item()
        gamma = gammas[gamma_idx]

        # rounded_num = np.round(gamma, 3)
        # formatted_num = ('{:.3f}'.format(rounded_num)).rstrip('0')
        # if formatted_num[-1] == '.':
        #     formatted_num += '0'
        # if rounded_num == 0 or rounded_num == 1:
        #     formatted_num = '{:.1f}'.format(rounded_num)
        gamma = int(np.round(gamma, 3)*1e2)
        
        setup = f"data_{datainfo['dataname']}_seed_{seed}_Penalty_power_Factor_{gamma}"
        print(setup)

        model_FT_name = f'./models/pNN_{setup}_FT.model'
        model_FT = torch.load(model_FT_name)

        args.powerestimator = 'power'
        args.powerbalance = float(gamma)
        loss_function = pNN.Lossfunction(args).to(args.DEVICE)

        # 计算 Hessian 矩阵的特征值
        eigenvalues = calculate_hessian_eigenvalues(model_FT, train_loader, loss_function)

        # 打印 Hessian 矩阵的特征值
        for n, eigvals in eigenvalues.items():
            # Sort eigenvalues by their absolute values in descending order
            top_eigvals = sorted(eigvals, key=lambda x: abs(x), reverse=True)[:3]
            print(f'Top 3 eigenvalues for Hessian of {n}:')
            for eigval in top_eigvals:
                values_target.append(abs(eigval).item())
                print(abs(eigval))

        values_ds.append(values_target)
    values.append(values_ds)

{'dataname': 'acuteinflammation', 'N_feature': 6, 'N_class': 2, 'N_train': 70, 'N_valid': 23, 'N_test': 25}
data_acuteinflammation_seed_0_Penalty_power_Factor_0


/home/kit/itec/qc0876/miniconda3/envs/train-model/lib/python3.12/site-packages/torch/autograd/__init__.py:266: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025824022/work/torch/csrc/autograd/engine.cpp:1177.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Top 3 eigenvalues for Hessian of act.rt_:
tensor(0.0005)
tensor(0.0001)
tensor(5.3744e-05)
Top 3 eigenvalues for Hessian of inv.rt_:
tensor(2.0211e-06)
tensor(1.6784e-06)
tensor(5.8042e-07)
Top 3 eigenvalues for Hessian of model.0-th pLayer.theta_:
tensor(0.0003)
tensor(0.0001)
tensor(7.1036e-05)
Top 3 eigenvalues for Hessian of model.1-th pLayer.theta_:
tensor(1.8986e-06)
tensor(5.6376e-07)
tensor(7.5750e-12)
data_acuteinflammation_seed_0_Penalty_power_Factor_0
Top 3 eigenvalues for Hessian of act.rt_:
tensor(0.0005)
tensor(0.0001)
tensor(5.3744e-05)
Top 3 eigenvalues for Hessian of inv.rt_:
tensor(2.0211e-06)
tensor(1.6784e-06)
tensor(5.8042e-07)
Top 3 eigenvalues for Hessian of model.0-th pLayer.theta_:
tensor(0.0003)
tensor(0.0001)
tensor(7.1036e-05)
Top 3 eigenvalues for Hessian of model.1-th pLayer.theta_:
tensor(1.8986e-06)
tensor(5.6376e-07)
tensor(7.5750e-12)
data_acuteinflammation_seed_0_Penalty_power_Factor_0
Top 3 eigenvalues for Hessian of act.rt_:
tensor(0.0005)
tensor(0.

In [9]:
values

[[[0.0005099023692309856,
   0.00010607696458464488,
   5.3744050092063844e-05,
   2.021123691520188e-06,
   1.6783894807304023e-06,
   5.804207034998399e-07,
   0.0003115413128398359,
   0.00013500098430085927,
   7.103587995516136e-05,
   1.8985890619660495e-06,
   5.637634785671253e-07,
   7.574994451142736e-12],
  [0.0005099023692309856,
   0.00010607696458464488,
   5.3744050092063844e-05,
   2.021123691520188e-06,
   1.6783894807304023e-06,
   5.804207034998399e-07,
   0.0003115413128398359,
   0.00013500098430085927,
   7.103587995516136e-05,
   1.8985890619660495e-06,
   5.637634785671253e-07,
   7.574994451142736e-12],
  [0.0005099023692309856,
   0.00010607696458464488,
   5.3744050092063844e-05,
   2.021123691520188e-06,
   1.6783894807304023e-06,
   5.804207034998399e-07,
   0.0003115413128398359,
   0.00013500098430085927,
   7.103587995516136e-05,
   1.8985890619660495e-06,
   5.637634785671253e-07,
   7.574994451142736e-12],
  [0.0005099023692309856,
   0.000106076964584

In [10]:
values = torch.tensor(values)
top_values, top_indices = values.topk(6, dim=-1)

# top_values will have the shape [13, 4, 3]
print("Shape of the resulting tensor:", top_values.shape)

Shape of the resulting tensor: torch.Size([13, 4, 6])


In [11]:
torch.save(top_values, '../RESULT/basline_hs_penalty.hessian')

In [12]:
for ds in range(num_dataset):
    print(top_values[ds])

tensor([[5.0990e-04, 3.1154e-04, 1.3500e-04, 1.0608e-04, 7.1036e-05, 5.3744e-05],
        [5.0990e-04, 3.1154e-04, 1.3500e-04, 1.0608e-04, 7.1036e-05, 5.3744e-05],
        [5.0990e-04, 3.1154e-04, 1.3500e-04, 1.0608e-04, 7.1036e-05, 5.3744e-05],
        [5.0990e-04, 3.1154e-04, 1.3500e-04, 1.0608e-04, 7.1036e-05, 5.3744e-05]])
tensor([[50.7685, 14.4575, 14.3998,  5.7923,  5.6392,  0.5188],
        [50.7685, 14.4575, 14.3998,  5.7923,  5.6392,  0.5188],
        [50.7685, 14.4575, 14.3998,  5.7923,  5.6392,  0.5188],
        [50.7685, 14.4575, 14.3998,  5.7923,  5.6392,  0.5188]])
tensor([[12.4113,  6.9662,  0.5588,  0.1934,  0.1599,  0.1407],
        [12.4113,  6.9662,  0.5588,  0.1934,  0.1599,  0.1407],
        [12.4113,  6.9662,  0.5588,  0.1934,  0.1599,  0.1407],
        [12.4113,  6.9662,  0.5588,  0.1934,  0.1599,  0.1407]])
tensor([[2.7486e+03, 1.8565e+03, 9.3346e+01, 1.2042e+00, 8.4394e-01, 2.6005e-01],
        [3.3233e+01, 1.9357e+01, 5.7486e+00, 4.9743e+00, 4.3362e+00, 3.8563